In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 2
BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE = 5 * 1e-4
PRIOR_FLAG = False
CUDA = torch.cuda.is_available()
PATH = 'ag-os-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE, PRIOR_FLAG)

In [3]:
from local_gibbs import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = True
# initialization
enc_z = Gibbs_z(K, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_skl import *
train(models, SKL_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\500 (13s),  symKL: 1153.611,  eubo: -357.287,  elbo: -1510.898,  ess: 1.136,  kl_eta_ex: 611.184,  kl_eta_in: 245.679,  kl_z_ex: 105.351,  kl_z_in: 86.878
epoch: 1\500 (12s),  symKL: 843.724,  eubo: -436.982,  elbo: -1280.706,  ess: 1.149,  kl_eta_ex: 479.922,  kl_eta_in: 251.911,  kl_z_ex: 85.672,  kl_z_in: 70.621
epoch: 2\500 (12s),  symKL: 644.427,  eubo: -470.070,  elbo: -1114.498,  ess: 1.167,  kl_eta_ex: 390.987,  kl_eta_in: 271.638,  kl_z_ex: 69.216,  kl_z_in: 55.913
epoch: 3\500 (12s),  symKL: 517.518,  eubo: -471.188,  elbo: -988.706,  ess: 1.202,  kl_eta_ex: 330.971,  kl_eta_in: 295.010,  kl_z_ex: 56.531,  kl_z_in: 46.084
epoch: 4\500 (14s),  symKL: 423.403,  eubo: -460.725,  elbo: -884.128,  ess: 1.240,  kl_eta_ex: 288.740,  kl_eta_in: 325.611,  kl_z_ex: 47.020,  kl_z_in: 39.617


KeyboardInterrupt: 

In [ ]:
torch.save(enc_eta.state_dict(), '../weights/enc-eta-%s' + PATH)
torch.save(oneshot_eta.state_dict(), '../weights/oneshot-eta-%s' + PATH)

In [ ]:
BATCH_SIZE_TEST = 50
Model_Params_Test = (N, K, D, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST)
obs, metric_step, reused = test(Eubo_cfz_os_eta, Data, Train_Params, Model_Params_Test, models)
(q_eta, _, q_z, _, _, _) = reused

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap